# Produce output data tree including MLP classifier 

Here we use the results of the trained MLP to classifier on our data. This is done via the SignalID class. The output is now defined by the tree data class and includes the observables and discriminatory variables needed for fitting.

In [1]:
gROOT->ProcessLine(".x LoadK2.C+");

%%%%%%%%%%%%%%%%%%%%%%%%%    BaseEventInfo
%%%%%%%%%%%%%%%%%%%%%%%%%    BaseRunInfo


Info in <ACLiC>: unmodified script has already been compiled and loaded


%%%%%%%%%%%%%%%%%%%%%%%%%    Weights
%%%%%%%%%%%%%%%%%%%%%%%%%    Bins
%%%%%%%%%%%%%%%%%%%%%%%%%    DataManager
%%%%%%%%%%%%%%%%%%%%%%%%%    LundReader
12121212121212121212121212 EventInfo
12121212121212121212121212 RunInfo
&&&&&&&&&&&&&&&&&&&&&&&&&&&& HSKinematics
&&&&&&&&&&&&&&&&&&&&&&&&&&&& Cuts
&&&&&&&&&&&&&&&&&&&&&&&&&&&& Combitorial
&&&&&&&&&&&&&&&&&&&&&&&&&&&& ParticleIter
&&&&&&&&&&&&&&&&&&&&&&&&&&&& Topology
&&&&&&&&&&&&&&&&&&&&&&&&&&&& FinalState
&&&&&&&&&&&&&&&&&&&&&&&&&&&& FiledTree
&&&&&&&&&&&&&&&&&&&&&&&&&&&& TreeData
&&&&&&&&&&&&&&&&&&&&&&&&&&&& TopoActionManager
&&&&&&&&&&&&&&&&&&&&&&&&&&&& ParticleCuts
&&&&&&&&&&&&&&&&&&&&&&&&&&&& VarsParticle
&&&&&&&&&&&&&&&&&&&&&&&&&&&& TreePrepParticle
&&&&&&&&&&&&&&&&&&&&&&&&&&&& MVASignalID
&&&&&&&&&&&&&&&&&&&&&&&&&&&& ParticleCutsManager
&&&&&&&&&&&&&&&&&&&&&&&&&&&& TreePrepManager
&&&&&&&&&&&&&&&&&&&&&&&&&&&& MVASignalIDManager
12121212121212121212121212 CLASTrigger


First create an object of your final state class. You may configure it if you want to limit the number of particles in an event or set to just run on generated simulation data.
This notebook is equivalent to the RunFSRootXXX.C macro which will be in the code directory.
Again just change XXX for your class name.

In [2]:
 K2 fs("Beam","ALL");
  // fs.SetGenerated(); //just analyse generated branch
  // fs.SetMaxParticles(10); //max number of particles of any 1 type
 

THSParticle::THSParticle, sorry particle -22 does not exist in ROOT PDG table
Charge ID particle are  -22 
 WARNING running HS::FinalState::AutoIter() you  should check the topology print output and make sure the displayed iterator is waht you want ..
Charge ID particle are  -22 
 WARNING running HS::FinalState::AutoIter() you  should check the topology print output and make sure the displayed iterator is waht you want ..
Charge ID particle are  -22 
 WARNING running HS::FinalState::AutoIter() you  should check the topology print output and make sure the displayed iterator is waht you want ..
Charge ID particle are  -22 
 WARNING running HS::FinalState::AutoIter() you  should check the topology print output and make sure the displayed iterator is waht you want ..
 Topology::Print() : 0
    particles = -10000 -22 10000 10000 
You can have any number of the following particles : 
 ALL 
The following particle are identified by pdg code : 
-22 
 ParticleIter::Print() 
     Type : 2 number 

Create an output tree. Here we will use a FiledTree object which is a wrapper class to a tree in a file. We will also create this as a smart pointer so we can delete it at the end by calling reset(), this will save the tree to disk. The Filed tree needs 2 strings ,1 a treename and the other the file name. You should give the full path for where you would like the files saved. It will overwrite any existing files. 

In [3]:
auto output = new HS::FiledTree("tt","Simtestmvamlpinc.root");
fs.FinalStateOutTree(output->Tree()); //connect ouput tree to final state branches

 HS::TreeData::Branches(TTree* tree) Type not defined fgIsA atomic_TClass_ptr branch will be omitted 


Here we add the cuts for each particle. This is done via a HS::Cuts object which gets propogated to each particle in each topology. To specify any type of cut behaviour you need to generate your own Cuts Method. Here we just use a predefined delta time cut but you could make your won class which has momentum dependence as well.

**Making your own cuts class** Here I use a new cuts class DeltaTimePCut. You can make your own cuts follwing the example notebook [DeltaTimePCut](MakeDeltaTimePCut.ipynb)

In [4]:
ParticleCutsManager cutsman;
auto beamCut=make_shared<DeltaTimePCut>(20,3); //7ns delta time cut
auto otherCut=make_shared<DeltaTimePCut>(20,3); //10ns cut for all other particles
cutsman.AddParticleCut("Beam",beamCut); //assign to manager
cutsman.SetDefaultCut(otherCut); //assign to manager
cutsman.ConfigureCuts(&fs);    //propogate the cuts to topologies in fs

Info ParticleCutsManager AddParticleCut set cut for 0
Info ParticleCutsManager AddParticleCut set cut for -22


Now register the cuts manager as a post topology function action. This will be applied after the Topo_0 etc functions when start times etc have been determined. 

In [5]:
fs.RegisterPostTopoAction(&cutsman);

**Particle Monitoring** here we are going to make seperate trees for each topology and only keep the detected particle information in each one.

In [6]:
auto treeman=new TreePrepManager("/work/g8dump/particlesTrain/MLPSim");
treeman->SetParticleVars("Beam",{"P","Time"}); //For beam only interesetef in energy and time
treeman->ConfigureTreeParticles(&fs); //propogate through topologies
//treeman.SetN(50000); //Number of events required for each topo. Number of train+test events must be less than this
fs.RegisterPostWorkAction(treeman); //register post-work i.e. after kinematics is called and all calculations

Adding Beam
    Current number of particles: 0
 Resized vectors to fill: 1, 1
Added 2 (Float_t) / 
Current particles: Beam  
Adding Proton
    Current number of particles: 1
 Resized vectors to fill: 2, 2
Added 6 (Float_t) / 
Current particles: Beam Proton  
Adding Kp
    Current number of particles: 2
 Resized vectors to fill: 3, 3
Added 6 (Float_t) / 
Current particles: Beam Proton Kp  
Adding Km
    Current number of particles: 3
 Resized vectors to fill: 4, 4
Added 6 (Float_t) / 
Current particles: Beam Proton Kp Km  
Setting branches...
Printing variables: 
            BeamP   0 / 0
            BeamTime   0 / 1
            ProtonP   1 / 0
            ProtonTh   1 / 1
            ProtonPhi   1 / 2
            ProtonTime   1 / 3
            ProtonEdep   1 / 4
            ProtonDeltaE   1 / 5
            KpP   2 / 0
            KpTh   2 / 1
            KpPhi   2 / 2
            KpTime   2 / 3
            KpEdep   2 / 4
            KpDeltaE   2 / 5
            KmP   3 / 0
            

Add the MLP respnose to the data stream. This just needs the directory created in the training routine.
Also apply a rough cut at MLP=0.1 to get rid of most of the background.

In [7]:
MVASignalIDManager sid("/work/g8dump/particlesTrain"); //directory from training
sid.SetParticleVars("Beam",{"P","Time"});
sid.ConfigureResults("MLP",&fs);
sid.SetCut(0.1); //Set classifier cut value, this will be applied to all events
fs.RegisterPostTopoAction(&sid);

Adding Beam
    Current number of particles: 0
 Resized vectors to fill: 1, 1
Added 2 (Float_t) / 
Current particles: Beam  
Adding Proton
    Current number of particles: 1
 Resized vectors to fill: 2, 2
Added 6 (Float_t) / 
Current particles: Beam Proton  
Adding Kp
    Current number of particles: 2
 Resized vectors to fill: 3, 3
Added 6 (Float_t) / 
Current particles: Beam Proton Kp  
Adding Km
    Current number of particles: 3
 Resized vectors to fill: 4, 4
Added 6 (Float_t) / 
Current particles: Beam Proton Kp Km  
Setting variable links...
Printing variables:  4
   number of vars 2
            BeamP   0 / 0
            BeamTime   0 / 1
   number of vars 6
            ProtonP   1 / 0
            ProtonTh   1 / 1
            ProtonPhi   1 / 2
            ProtonTime   1 / 3
            ProtonEdep   1 / 4
            ProtonDeltaE   1 / 5
   number of vars 6
            KpP   2 / 0
            KpTh   2 / 1
            KpPhi   2 / 2
            KpTime   2 / 3
            KpEdep   2 /

Now we configure the input data. Here we use a ROOT file presaved in HSDATA format. It is possible to analyse other dataformats via the DataManager class.
Now analyse all the data in the 1.9GeV peak setting ~ 530M events.

In [8]:
 //create datamanager
  auto dm=std::make_shared<DataManager>();

  //And make a chain of data files
  TChain chain("HSParticles");
 // chain.Add("/home/dglazier/fastdata/g8/flist_048200.dat.root");
  chain.Add("/home/dglazier/fastdata/g8/rec_edgen_.root");

 // chain.Add("/w/work1/jlab/hallb/HSg8/g8sim_2k_b/rec_edgen_.root");
   dm->InitChain(&chain);

  //connect FinalState to Data by moving the pointer
  fs.SetDataManager(dm);

DataManager::InitReader set branch 0x7fe2e6341d20 0x7fe2e6341d20
DataManager::InitReader set branch Generated 0x7fe2e6341d38 
DataManager::InitReader EventInfo
DataManager::InitReader RunInfo


Info in <DataManager::InitChain>:  Will proceess all 1 files in chain


Analyse all the events. This took around 8 hours with the call to MLP. BDT 5-10 x slower

In [9]:
cout<<"Number of Events to process "<<chain.GetEntries()<<endl;

fs.ProcessData(); //No number give, analyse all events in chain
cout<<"Done "<<dm->GetEntry()<< " events "<<endl;

Number of Events to process 9895530
 ProcessData event 100000
 ProcessData event 200000
 ProcessData event 300000
 ProcessData event 400000
 ProcessData event 500000
 ProcessData event 600000
 ProcessData event 700000
 ProcessData event 800000
 ProcessData event 900000
 ProcessData event 1000000
 ProcessData event 1100000
 ProcessData event 1200000
 ProcessData event 1300000
 ProcessData event 1400000
 ProcessData event 1500000
 ProcessData event 1600000
 ProcessData event 1700000
 ProcessData event 1800000
 ProcessData event 1900000
 ProcessData event 2000000
 ProcessData event 2100000
 ProcessData event 2200000
 ProcessData event 2300000
 ProcessData event 2400000
 ProcessData event 2500000
 ProcessData event 2600000
 ProcessData event 2700000
 ProcessData event 2800000
 ProcessData event 2900000
 ProcessData event 3000000
 ProcessData event 3100000
 ProcessData event 3200000
 ProcessData event 3300000
 ProcessData event 3400000
 ProcessData event 3500000
 ProcessData event 3600000
 

And remember to save the tree!

In [10]:
delete output;

In [11]:
%jsroot on

In [12]:
TCanvas c3;
c3.Divide(2,2);
c3.cd(1);
treeman->GetPrep(0)->Tree()->Draw("BeamP:BeamTime>>h1(100,-10,10,100,0.4)","MLP>0.95","col1");
c3.cd(2);
treeman->GetPrep(0)->Tree()->Draw("ProtonP:ProtonTime>>h2(100,-10,10,100,0,4)","MLP>0.95","col1");
c3.cd(3);
treeman->GetPrep(0)->Tree()->Draw("KpP:KpTime>>h3(100,-10,10,100,0,4)","MLP>0.95","col1");
c3.cd(4);
treeman->GetPrep(0)->Tree()->Draw("KmP:KmTime>>h4(100,-10,10,100,0,4)","MLP>0.95","col1");


input_line_186:2:3: error: use of undeclared identifier 'c3'
 (c3.Divide(2, 2))
  ^
Error in <HandleInterpreterException>: Error evaluating expression (c3.Divide(2, 2)).
Execution of your code was aborted.


In [13]:
c3.Draw();

input_line_188:2:3: error: use of undeclared identifier 'c3'
 (c3.Draw())
  ^
Error in <HandleInterpreterException>: Error evaluating expression (c3.Draw()).
Execution of your code was aborted.


**When ready delete the treeman so trees as saved to the output directory**

In [14]:
delete treeman;